In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.95, 0.91, 0.87, 0.87) seconds, 3.6 seconds total
Sampling took (1.1, 1.2, 1.1, 0.97) seconds, 4.3 seconds total

                    Mean     MCSE   StdDev        5%       50%       95%    N_Eff  N_Eff/s    R_hat
lp__             9.3e+02  7.8e-02  3.1e+00   9.2e+02   9.3e+02   9.3e+02  1.6e+03  3.7e+02  1.0e+00
accept_stat__   

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE       MCSE         ESS   
         lp__  928.101898750  3.1083354479 0.03475224679 0.0793423249 1534.77925
accept_stat__    0.941430719  0.0819585915 0.00091632491 0.0013628982 2000.00000
   stepsize__    0.046786400  0.0041829256 0.00004676653 0.0004705865   79.00988
  treedepth__    5.913875000  0.4449519436 0.00497471396 0.0115909908 1473.61860
 n_leapfrog__   74.475000000 33.7208913333 0.37701102642 0.8157237603 1708.88118
  divergent__    0.000000000  0.0000000000 0.00000000000 0.0000000000        NaN
     energy__ -921.638855000  4.0156429604 0.04489625316 0.0936420591 1838.94302
            a    1.112384857  0.6988313916 0.00781317248 0.0127413710 2000.00000
           bp    0.259945793  0.0757893428 0.00084735061 0.0013489914 2000.00000
  a_society.1   -0.200470175  0.2376460349 0.00265696344 0.0035954955 2000

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*